# Pipelines: Metaflow model training

##  Install dependencies

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

## Set username

In [3]:
# Set username for workflows
import os
os.environ["USERNAME"] = "eduardo"

In [12]:
%%writefile metaflow_trainingflow.py
from metaflow import FlowSpec, Parameter, step
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pickle
import os


class TrainingFlow(FlowSpec):
    # Define the parameters for the flow
    max_depth = Parameter('max_depth', default=2, help='Max depth of the random forest classifier')
    n_estimators = Parameter('n_estimators', default=100, help='Number of estimators for the random forest classifier')
    random_state = Parameter('random_state', default=0, help='Random state for the random forest classifier')

    @step
    def start(self):
        # Start the flow
        self.next(self.ingest_data)

    @step
    def ingest_data(self):
        from sklearn.datasets import load_iris

        # Load the iris dataset
        iris = load_iris()

        #pylint: disable=no-member
        self.X = iris.data
        self.y = iris.target
        #pylint: enable=no-member

        self.next(self.split_data)

    @step
    def split_data(self):
        #Split the data into train and test
        # TODO: WRITE YOUR CODE HERE
        self.X_train, self.X_test, self.y_train,self. y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42, stratify=self.y)

        self.next(self.train)
    @step

    def train(self):
        # Definir el modelo usando los parámetros de la clase TrainingFlow
        self.model = RandomForestClassifier(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            random_state=self.random_state
        )

        # Entrenar el modelo con los datos de entrenamiento
        self.model.fit(self.X_train, self.y_train)

        self.next(self.show_metrics)
    @step
    def show_metrics(self):
        # Print some metrics
        # TODO: WRITE YOUR CODE HERE
        # Hacer predicciones en el conjunto de prueba
        y_pred = self.model.predict(self.X_test)

        # Calcular métricas
        accuracy = accuracy_score(self.y_test, y_pred)
        precision = precision_score(self.y_test, y_pred, average='macro')  # Cambia a 'weighted' si hay desbalance
        recall = recall_score(self.y_test, y_pred, average='macro')
        f1 = f1_score(self.y_test, y_pred, average='macro')
        self.next(self.register_model)
    @step
    def register_model(self):
        # Save the model in a pickle file in local storage
        # TODO: WRITE YOUR CODE HERE
        # Definir el nombre del archivo
        model_filename = "trained_model.pkl"

        # Guardar el modelo en un archivo pickle
        with open(model_filename, "wb") as file:
            pickle.dump(self.model, file)
        self.next(self.end)
    @step
    def end(self):
        pass

if __name__ == '__main__':
    TrainingFlow()

Writing metaflow_trainingflow.py


In [13]:
!python metaflow_trainingflow.py run --max_depth 2 --n_estimators 100 --random_state 0

Metaflow 2.13.9 executing TrainingFlow for user:eduardo
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
2025-03-10 12:23:21.666 Workflow starting (run-id 1741609401665815):
2025-03-10 12:23:21.674 [1741609401665815/start/1 (pid 5124)] Task is starting.
2025-03-10 12:23:23.618 [1741609401665815/start/1 (pid 5124)] Task finished successfully.
2025-03-10 12:23:23.623 [1741609401665815/ingest_data/2 (pid 5139)] Task is starting.
2025-03-10 12:23:25.553 [1741609401665815/ingest_data/2 (pid 5139)] Task finished successfully.
2025-03-10 12:23:25.558 [1741609401665815/split_data/3 (pid 5154)] Task is starting.
2025-03-10 12:23:27.419 [1741609401665815/split_data/3 (pid 5154)] Task finished successfully.
2025-03-10 12:23:27.424 [1741609401665815/train/4 (pid 5165)] Task is starting.
2025-03-10 12:23:29.890 [1741609401665815/train/4 (pid 5165)] Task finished successfully.
2025-03-10 12:23:29.895 [1741609401665815/show_metric